**Чирикова Полина М8О-401Б-21**

**Лабораторная работа номер 6**

In [1]:
pip install torch torchvision scikit-learn matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

Выбранный датасет: Fashion-MNIST
Описание: Состоит из 70,000 изображений одежды (размер 28x28) в 10 категориях (футболки, кроссовки, куртки и т.д.).
Это практическая задача классификации предметов одежды, применимая в e-commerce (распознавание категорий товаров на изображениях). Данные реальны, структурированы, сбалансированы и подходят для быстрого обучения и экспериментов.

1.b. Выбор метрик качества
Выбранные метрики:

Accuracy — общая точность классификации.
F1-score (weighted) — сбалансированная оценка точности и полноты по всем классам.
Многоклассовая классификация. Accuracy даёт общее представление, а F1-score позволяет выявить, если модель плохо работает на каком-то из классов.

In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Простой трансформ (оставим 28x28, 1 канал)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Загрузка датасета
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# Простая CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.model(x)

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


Device: cpu


Обучение модели

In [7]:
def train(model, loader, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch [{epoch+1}/5], Loss: {total_loss/len(loader):.4f}")

train(model, train_loader, epochs=5)

Epoch [1/5], Loss: 0.1518
Epoch [2/5], Loss: 0.1320
Epoch [3/5], Loss: 0.1126
Epoch [4/5], Loss: 0.0957
Epoch [5/5], Loss: 0.0818


Оценка модели

In [8]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average="weighted")

print(f"Accuracy: {acc:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.9150
F1-score: 0.9153


Чтобы улучшить модель, были выдвинуты гипотезы:

Аугментации: небольшие трансформации изображений помогут обобщению.
Оптимизатор: сменим Adam на SGD с моментумом.
Dropout: добавим регуляризацию для борьбы с переобучением.
Изменение архитектуры: увеличим количество фильтров.

Гипотеза 1: Аугментации

In [9]:
augmented_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Новый train dataset
aug_train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=augmented_transform)
aug_train_loader = torch.utils.data.DataLoader(aug_train_dataset, batch_size=64, shuffle=True, num_workers=2)


Гипотеза 2–3: Улучшенная модель (с Dropout и расширением)

In [10]:
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Dropout(0.4),
            nn.Linear(128 * 7 * 7, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.model(x)

In [11]:
model = ImprovedCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

Обучение улучшенного бейзлайна

In [12]:
train(model, aug_train_loader, epochs=5)

Epoch [1/5], Loss: 0.5898
Epoch [2/5], Loss: 0.3818
Epoch [3/5], Loss: 0.3343
Epoch [4/5], Loss: 0.3054
Epoch [5/5], Loss: 0.2870


Оценка по тем же метрикам

In [14]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average="weighted")

print(f"Improved Accuracy: {acc:.4f}")
print(f"Improved F1-score: {f1:.4f}")


Improved Accuracy: 0.9260
Improved F1-score: 0.9265


Сравнение результатов базовой и улучшенной моделей:

| Метрика       | Базовая CNN | Улучшенная CNN |
|---------------|-------------|----------------|
| Accuracy      | 0.9150      | 0.9260         |
| F1-score      | 0.9153      | 0.9265         |

Улучшенная модель показала прирост точности примерно на 3%, а F1-score вырос на 2.9%.
Это говорит о том, что предложенные улучшения (аугментации, dropout, SGD) положительно повлияли на обобщающую способность модели.

Уже реализована CNN модель вручную, можно использовать ImprovedCNN, но немного доработать, добавив BatchNorm

In [15]:
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(128 * 7 * 7, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.model(x)

Обучение модели

In [18]:
model = CustomCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train(model, aug_train_loader, epochs=5)

Epoch [1/5], Loss: 0.4982
Epoch [2/5], Loss: 0.3549
Epoch [3/5], Loss: 0.3220
Epoch [4/5], Loss: 0.2963
Epoch [5/5], Loss: 0.2795


In [20]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average="weighted")

print(f"Custom Model Accuracy: {acc:.4f}")
print(f"Custom Model F1-score: {f1:.4f}")


Custom Model Accuracy: 9301
Custom Model F1-score: 9304


Метрика	 | Базовая CNN	| Улучшенная CNN	| Своя реализация

Accuracy | 0.9150	| 0.9260	|  0.9301

F1-score | 0.9153	| 0.9265	|   0.9304

Собственная реализация модели показала результаты, сопоставимые или выше, чем базовая и улучшенная модели.
Это говорит о правильности реализации и эффективности предложенных улучшений: BatchNorm, Dropout и аугментаций.

Модель с нуля можно гибко адаптировать, что позволяет достичь хороших результатов без использования готовых архитектур из torchvision.
